# Assessment Problem
Write a Python script that processes sensor data from a zip file (`sensor_data_2025.zip`). The script should:
1. Extract data from nested city zip files (e.g., `munich_data.zip`, `tokyo_data.zip`), reading ``readings.csv`` and `station_info.json` from each.
2. Add a `city` column to each DataFrame with the city's name.
3. Combine the data into two main DataFrames: one for readings and one for station info.
4. Merge these two DataFrames on `station_id`.
5. Clean the merged data by keeping only `active` stations and removing rows with missing `temperature_celsius` or `pm2_5`.
6. Calculate the average temperature and PM2.5 for each city.
7. Save the city summary as a CSV file named `city_summary_report.csv` with columns `city`, `average_temperature`, and `average_pm2_5`.

## TASK 1: Load and combine data

Go through each city's zip file, read the `readings.csv` and `station_info.json` files, add a 'city' column to each DataFrame, and combine them into `all_readings_df` and `all_station_info_df`.


**Reasoning**:
TASK 1 requires extracting data from nested zip files, processing them, and combining them into two dataframes. This involves using the zipfile library and pandas for data manipulation.



In [1]:
#importing the libraries
import pandas as pd 
import zipfile
import numpy
#unziping the files
def unzip(filename,folder_name):
    with zipfile.ZipFile(filename,'r') as zf:
        inner_file=zf.namelist()
        zf.extractall(folder_name)
        return inner_file
inner_file=unzip('sensor_data_2025.zip','Final_Assessment')
unzip('Final_Assessment/munich_data.zip','munich_data')
unzip('Final_Assessment/tokyo_data.zip','tokyo_data')
unzip('Final_Assessment/sydney_data.zip','sydney_data')
munich_csv=pd.read_csv('munich_data/readings.csv')
munich_csv['city']='munich'
sydney_csv=pd.read_csv('sydney_data/readings.csv')
sydney_csv['city']='sydney'
tokyo_csv=pd.read_csv('tokyo_data/readings.csv')
tokyo_csv['city']='Tokyo'
munich_json=pd.read_json('munich_data/station_info.json')
munich_json['city']='munich'
tokyo_json=pd.read_json('tokyo_data/station_info.json')
tokyo_json['city']='tokyo'
sydney_json=pd.read_json('sydney_data/station_info.json')
sydney_json['city']='sydney'
all_reading_df=pd.concat([munich_csv,tokyo_csv,sydney_csv])
all_station_info_df=pd.concat([munich_json,tokyo_json,sydney_json])

## TASK 2: Merge dataframes

Merge the `all_readings_df` and `all_station_info_df` into a single DataFrame using the `station_id` column.


**Reasoning**:
Merge the two dataframes `all_readings_df` and `all_station_info_df` on the `station_id` column and display the head of the resulting dataframe to verify the merge.



In [2]:
main_df=all_reading_df.merge(all_station_info_df,on='station_id')
main_df.head()

,station_id,timestamp,temperature_celsius,humidity_percent,pm2_5,city_x,location,status,deployment_year,city_y
0,1001,2025-02-04 23:00:00,14.17,77.11,22.80,munich,"{'latitude': 48.116274, 'longitude': 11.563626}",active,2024,munich
1,1001,2025-01-06 16:00:00,2.66,60.41,29.61,munich,"{'latitude': 48.116274, 'longitude': 11.563626}",active,2024,munich
2,1001,2025-01-01 13:00:00,0.31,53.84,42.90,munich,"{'latitude': 48.116274, 'longitude': 11.563626}",active,2024,munich
3,1001,2025-01-20 11:00:00,14.97,80.40,NaN,munich,"{'latitude': 48.116274, 'longitude': 11.563626}",active,2024,munich
4,1001,2025-02-12 06:00:00,14.22,78.14,62.48,munich,"{'latitude': 48.116274, 'longitude': 11.563626}",active,2024,munich


## TASK 3: Clean data

Filter the merged DataFrame to keep only 'active' stations and remove rows with missing values in 'temperature_celsius' or 'pm2_5'.


**Reasoning**:
Filter the merged dataframe to keep only active stations and remove rows with missing values in the specified columns.



In [3]:
cleaned_df=main_df[main_df['status']=='active']
cleaned_df.dropna(subset=['temperature_celsius','pm2_5'],how='any',inplace=True)

C:\Users\Baby.Chandran\AppData\Local\Temp\ipykernel_21656\3921023914.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df.dropna(subset=['temperature_celsius','pm2_5'],how='any',inplace=True)


## TASK 4: Analyze data

Calculate the average 'temperature_celsius' and 'pm2_5' for each city.


**Reasoning**:
Calculate the mean temperature and PM2.5 for each city by grouping the cleaned_df by city and applying the mean aggregation. Then display the head of the resulting dataframe.



In [4]:
cleaned_df_pm2=cleaned_df.groupby(['city_y'],as_index=False)['pm2_5'].mean()
cleand_df_temp=cleaned_df.groupby(['city_y'],as_index=False)['temperature_celsius'].mean()
result_df=cleand_df_temp.merge(cleaned_df_pm2,on=['city_y'])
result_df.head()

,city_y,temperature_celsius,pm2_5
0,munich,7.550826,40.038595
1,sydney,24.261897,39.060256
2,tokyo,12.685729,39.741027


## TASK 5: Save report

Save the city summary as a CSV file named `city_summary_report.csv` with the specified columns.


**Reasoning**:
Reset the index, rename the city column, and save the city summary DataFrame to a CSV file.



In [5]:
resut=result_df.to_csv('city_summary_report.csv',index=False)